In [221]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from airquality.data.read_data import read_obs, read_targets

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
'''
headers_path = '/Users/b.yc0006/Cloud/BCNAirQualityDatathon/data/raw/headers_mod.csv'
headers_obs = '/Users/b.yc0006/Cloud/BCNAirQualityDatathon/data/raw/headers_obs.csv'
targets_path = '/Users/b.yc0006/Cloud/BCNAirQualityDatathon/data/raw/targets.csv'
stations_path = '/Users/b.yc0006/Cloud/BCNAirQualityDatathon/data/raw/stations.csv'
'''
all_models_path = '/Users/b.yc0006/Cloud/BCNAirQualityDatathon/data/raw/all_models.csv'
submission_path = '/Users/b.yc0006/Cloud/BCNAirQualityDatathon/data/submissions/'

In [59]:
data = pd.read_csv(all_models_path).iloc[:,1:]

# Exploring

In [49]:
data.head()

,lon,lat,day,hour,AirPollutant,Concentration,year,station
0,2.1331,41.3788,2013-03-31,00:00:00,NO2,10.17750,2013,ES1396A
1,2.1331,41.3788,2013-03-31,01:00:00,NO2,9.07109,2013,ES1396A
2,2.1331,41.3788,2013-03-31,02:00:00,NO2,7.36639,2013,ES1396A
3,2.1331,41.3788,2013-03-31,03:00:00,NO2,8.24546,2013,ES1396A
4,2.1331,41.3788,2013-03-31,04:00:00,NO2,9.76113,2013,ES1396A


In [50]:
data.tail()

,lon,lat,day,hour,AirPollutant,Concentration,year,station
367901,2.18742,41.3864,2015-09-16,19:00:00,NO2,31.1346,2015,ES1679A
367902,2.18742,41.3864,2015-09-16,20:00:00,NO2,36.4066,2015,ES1679A
367903,2.18742,41.3864,2015-09-16,21:00:00,NO2,39.9240,2015,ES1679A
367904,2.18742,41.3864,2015-09-16,22:00:00,NO2,37.2978,2015,ES1679A
367905,2.18742,41.3864,2015-09-16,23:00:00,NO2,36.2155,2015,ES1679A


In [30]:
targets = pd.read_csv(targets_path)
stations = pd.read_csv(stations_path)

In [32]:
print targets['target'].mean()
targets.head()

0.0383233532934


,date,station,target
0,2015-01-03,ES0691A,0
1,2015-01-03,ES1396A,0
2,2015-01-03,ES1438A,0
3,2015-01-03,ES1480A,0
4,2015-01-03,ES1679A,0


In [36]:
targets['target'] = 0
submission = targets['target']

In [40]:
submission

0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25     0
26     0
27     0
28     0
29     0
      ..
805    0
806    0
807    0
808    0
809    0
810    0
811    0
812    0
813    0
814    0
815    0
816    0
817    0
818    0
819    0
820    0
821    0
822    0
823    0
824    0
825    0
826    0
827    0
828    0
829    0
830    0
831    0
832    0
833    0
834    0
Name: target, Length: 835, dtype: int64

In [42]:
submission.to_csv(submission_path+'submission_zeros.csv', header=True, index=False)

In [12]:
stations.head()

,code,name,lat,lon,height,city,country
0,ES1480A,Barcelona (Gràcia-Sant Gervasi),41.3987,2.15339,57,Barcelona,España
1,ES1856A,Barcelona (Vall d'Hebron),41.4261,2.14799,136,Barcelona,España
2,ES1679A,Ciutadella,41.3864,2.18742,7,Barcelona,España
3,ES1438A,Barcelona (Eixample),41.3853,2.15382,26,Barcelona,España
4,ES1396A,Barcelona,41.3788,2.13310,35,Barcelona,España


### Keras LSTM

In [233]:
from airquality.models.LSTM_keras import LSTM_K
from airquality.data.prepare_data import create_model_matrix, create_ts_df

In [234]:
data_obs = read_obs()
data_targets = read_targets()

In [235]:
target_cols = data_targets.columns[:-1]

In [236]:
data_obs.head()

,ES0691A,ES1396A,ES1438A,ES1480A,ES1679A,ES1856A,ES1992A,datetime,date
0,55,39.0,51.0,54.0,48,11.0,19.0,2013-01-01 01:00:00,2013-01-01
1,48,30.0,50.0,47.0,62,11.0,13.0,2013-01-01 02:00:00,2013-01-01
2,53,33.0,58.0,38.0,64,13.0,15.0,2013-01-01 03:00:00,2013-01-01
3,46,25.0,43.0,30.0,42,5.0,11.0,2013-01-01 04:00:00,2013-01-01
4,31,21.0,38.0,20.0,24,4.0,6.0,2013-01-01 05:00:00,2013-01-01


In [248]:
model_matrix = create_model_matrix(data_obs, target_cols=target_cols)
model_matrix.head()

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [241]:
def tt_split(data, train_size, target_cols):
    test_size = len(data) - train_size
    trainX, testX = data[:train_size].drop(target_cols, axis=1).values, data[train_size:].drop(target_cols, axis=1).values
    trainY, testY = data[target_cols][:train_size].values, data[target_cols][train_size:].values
    return trainX.reshape([-1,trainX.shape[1]]), testX.reshape([-1,trainX.shape[1]]), trainY, testY

In [242]:
# split into train and test sets
train_size = int(len(model_matrix)-1)
print train_size, len(model_matrix)
trainX, testX, trainY, testY = tt_split(model_matrix, train_size, target_cols)
print trainX.shape, testX.shape
print trainY.shape, testY.shape

22597 22598
(22597, 9) (1, 9)
(22597, 7) (1, 7)


In [243]:
def reshape_to_keras(data, seq_length):
    data = data.reshape([-1, seq_length, data.shape[1]])
    return data

In [244]:
seq_length = 1
trainX = reshape_to_keras(trainX, seq_length)
testX = reshape_to_keras(testX, seq_length)
print trainX.shape, testX.shape
print trainY.shape, testY.shape

(22597, 1, 9) (1, 1, 9)
(22597, 7) (1, 7)


In [246]:
param_dict = {
    'batch_size': 10,
    'seq_length': seq_length,
    'size': trainX.shape[2],
    'hidden_units': 50,
    'num_layers': 1,
    'dense_units': trainY.shape[1],
    'epochs': 5,
}

lstm = LSTM_K(**param_dict)

In [247]:
lstm.fit(trainX, trainY)

Epoch 1/5


TypeError: float() argument must be a string or a number

In [ ]:
lstm.predict(testX)

In [218]:
testY

array([[42.3477, 56.9491, 70.1963, 69.5196, 38.2253, 36.2155, 87.4319]])